In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
df=pd.read_csv('/kaggle/input/analyticsvidya-jobathon-5-aug-2022/analyticsvidya_jobathon_5_aug_2022_train_F3fUq2S.csv')
df_test=pd.read_csv('/kaggle/input/analyticsvidya-jobathon-5-aug-2022/analyticsvidya_jobathon_5_aug_2022_test_Bk2wfZ3.csv')
sample_sub=pd.read_csv('/kaggle/input/analyticsvidya-jobathon-5-aug-2022/analyticsvidya_jobayhon_5_aug_2022_sample_submission_LJ2N3ZQ.csv')

In [3]:
df.drop(['campaign_id'],axis=1,inplace=True)
df_test.drop(['campaign_id'],axis=1,inplace=True)

In [4]:
y=df['click_rate']
X=df.drop('click_rate',axis=1)

In [5]:
num_cols=['subject_len','body_len','mean_paragraph_len','no_of_CTA','mean_CTA_len','target_audience']
cat_cols=[c for c in X.columns if c not in num_cols and c not in ['product','times_of_day','day_of_week']]
actual_cat_cols=['times_of_day','day_of_week']

In [6]:
#polynomial  num cols + cat cols
#Normalizer
#kfold = 5
#xgboost best 
#optuna 0.5910286425696581

from sklearn.linear_model import LinearRegression,Ridge
from sklearn.metrics import r2_score
from sklearn import preprocessing,model_selection
from xgboost import XGBRegressor

df_predictions=[]
models=[]
r2_scores_train=[]
r2_scores_val=[]
df_test_1=df_test.copy()

poly=preprocessing.PolynomialFeatures(1,interaction_only=True)
X_poly=pd.DataFrame(poly.fit_transform(X[num_cols+cat_cols]))
df_test_1_poly=pd.DataFrame(poly.transform(df_test_1[num_cols+cat_cols]))

X_poly=pd.concat([X_poly.reset_index(drop=True),X[actual_cat_cols].reset_index(drop=True)],axis=1)
df_test_1_poly=pd.concat([df_test_1_poly.reset_index(drop=True),df_test_1[actual_cat_cols].reset_index(drop=True)],axis=1)

kf=model_selection.KFold(5,shuffle=True,random_state=1)
for knum,(train_idx,test_idx) in enumerate(kf.split(X_poly)):
    X_train,X_val=X_poly.iloc[train_idx],X_poly.iloc[test_idx]
    y_train,y_val=y[train_idx],y[test_idx]
    df_test_1=df_test_1_poly.copy()
    
    
    
   
    ord_enc=preprocessing.OrdinalEncoder()
    
    X_train_ord=pd.DataFrame(ord_enc.fit_transform(X_train[actual_cat_cols]))
    X_val_ord=pd.DataFrame(ord_enc.transform(X_val[actual_cat_cols]))
    df_test_1_ord=pd.DataFrame(ord_enc.transform(df_test_1[actual_cat_cols]))
    
    X_train_ord.index=X_train.index
    X_val_ord.index=X_val.index
    df_test_1_ord.index=df_test_1.index
    
    
    
    
    std_scl=preprocessing.Normalizer()
    X_train_rem=pd.DataFrame(std_scl.fit_transform(X_train.drop(actual_cat_cols,axis=1)))
    X_val_rem=pd.DataFrame(std_scl.transform(X_val.drop(actual_cat_cols,axis=1)))
    df_test_1_rem=pd.DataFrame(std_scl.transform(df_test_1.drop(actual_cat_cols,axis=1)))
    
    
    
    X_train=pd.concat([X_train_rem.reset_index(drop=True),X_train_ord.reset_index(drop=True)],axis=1,ignore_index=True)
    X_val=pd.concat([X_val_rem.reset_index(drop=True),X_val_ord.reset_index(drop=True)],axis=1,ignore_index=True)
    df_test_1=pd.concat([df_test_1_rem.reset_index(drop=True),df_test_1_ord.reset_index(drop=True)],axis=1,ignore_index=True)
    

    
    new_params={'learning_rate': 0.015119207349205814,
     'reg_lambda': 1.2652822281260555e-05,
     'reg_alpha': 1.9346755925423264e-05,
     'subsample': 0.886572540286282,
     'colsample_bytree': 0.5999537586785572,
     'max_depth': 6}
    model=XGBRegressor(n_jobs=-1,random_state=1
                        #,gpu_id=0,tree_method='gpu_hist',predictor='gpu_predictor'
                        ,**new_params
                      ,early_stopping_rounds=300
                      ,n_estimators=7000)
    
    model.fit(X_train,y_train,eval_set=[(X_val,y_val)],verbose=1000)
    y_pred=model.predict(X_val)
    
    
    print('R2 Score : Training set = ',r2_score(y_train,model.predict(X_train)),'    Validation set = ',r2_score(y_val,y_pred))
    
    df_predictions.append(model.predict(df_test_1))
    #models.append(model)
    r2_scores_train.append(r2_score(y_train,model.predict(X_train)))
    r2_scores_val.append(r2_score(y_val,y_pred))
    
print('Avg r2 score:  Training : ',np.mean(r2_scores_train),'    Validation set = ',np.mean(r2_scores_val))

#predictions=[]
#for i in range(len(models)):
#    predictions.append(models[i].predict(X))
    
#print('Final r2 score:  Training : ',r2_score(y,np.mean(predictions,axis=0)))
final_prediction1=np.mean(df_predictions,axis=0).tolist()


In [7]:
#final_prediction1.tolist()

In [8]:
#polynomial  num cols + cat cols
#Normalizer
#kfold = 5
#xgboost 
# Score =0.5403177047598372 ,fold=5, - Absolute normal only ordinal enc 'time_of_day'
# Score=0.5805807377193353  ,fold=4, - ohe hot encode cols- actual_cat_cols=['times_of_day','day_of_week']
# Score=0.58394352213935    ,fold=5  -same as above
#Score=0.6059578883882286   ,fold=5  -same as above, with optuna

#Score=0.5752140844080066   ,fold=5  - booster : 'dart', rest same as above


from sklearn.linear_model import LinearRegression,Ridge
from sklearn.metrics import r2_score
from sklearn import preprocessing,model_selection
from xgboost import XGBRegressor

df_predictions=[]
models=[]
r2_scores_train=[]
r2_scores_val=[]
df_test_1=df_test.copy()

num_cols=['subject_len','body_len','mean_paragraph_len','no_of_CTA','mean_CTA_len','target_audience']
cat_cols=[c for c in X.columns if c not in num_cols and c not in ['product','times_of_day','day_of_week']]
actual_cat_cols=['times_of_day','day_of_week']


kf=model_selection.KFold(5,shuffle=True,random_state=1)
for knum,(train_idx,test_idx) in enumerate(kf.split(X_poly)):
    X_train,X_val=X.iloc[train_idx],X.iloc[test_idx]
    y_train,y_val=y[train_idx],y[test_idx]
    df_test_1=df_test.copy()
    
    #ord_enc=preprocessing.OrdinalEncoder()
    #X_train[['times_of_day']]=pd.DataFrame(ord_enc.fit_transform(X_train[['times_of_day']]))
    #X_val[['times_of_day']]=pd.DataFrame(ord_enc.transform(X_val[['times_of_day']]))
    #df_test_1[['times_of_day']]=pd.DataFrame(ord_enc.transform(df_test_1[['times_of_day']]))
    
    ohe=preprocessing.OneHotEncoder(sparse=False)
    X_train_ohe=pd.DataFrame(ohe.fit_transform(X_train[actual_cat_cols]))
    X_val_ohe=pd.DataFrame(ohe.transform(X_val[actual_cat_cols]))
    df_test_1_ohe=pd.DataFrame(ohe.transform(df_test_1[actual_cat_cols]))
    
   
    #X_train=np.log1p(X_train)
    #X_val=np.log1p(X_val)
    #df_test_1=np.log1p(df_test_1)
    
    X_train=pd.concat([X_train.drop(actual_cat_cols,axis=1).reset_index(drop=True),X_train_ohe.reset_index(drop=True)],axis=1,ignore_index=True)
    X_val=pd.concat([X_val.drop(actual_cat_cols,axis=1).reset_index(drop=True),X_val_ohe.reset_index(drop=True)],axis=1,ignore_index=True)
    df_test_1=pd.concat([df_test_1.drop(actual_cat_cols,axis=1).reset_index(drop=True),df_test_1_ohe.reset_index(drop=True)],axis=1,ignore_index=True)
    

    #model=LinearRegression()
    new_params={'learning_rate': 0.03488164266932342,
     'reg_lambda': 0.14638377598662689,
     'reg_alpha': 0.002118079241624242,
     'subsample': 0.8058916328245683,
     'colsample_bytree': 0.5389818688250179,
     'max_depth': 7}
    model=XGBRegressor(n_jobs=-1,random_state=1
                        #,gpu_id=0,tree_method='gpu_hist',predictor='gpu_predictor'
                        ,**new_params
                      ,early_stopping_rounds=300
                      ,n_estimators=7000
                      #,enable_categorical=True
                      )
    model.fit(X_train,y_train,eval_set=[(X_val,y_val)],verbose=1000)
    y_pred=model.predict(X_val)
    
    
    print('R2 Score : Training set = ',r2_score(y_train,model.predict(X_train)),'    Validation set = ',r2_score(y_val,y_pred))
    
    df_predictions.append(model.predict(df_test_1))
    models.append(model)
    r2_scores_train.append(r2_score(y_train,model.predict(X_train)))
    r2_scores_val.append(r2_score(y_val,y_pred))
    
print('Avg r2 score:  Training : ',np.mean(r2_scores_train),'    Validation set = ',np.mean(r2_scores_val))

#predictions=[]
#for i in range(len(models)):
#    predictions.append(models[i].predict(X))
    
#print('Final r2 score:  Training : ',r2_score(y,np.mean(predictions,axis=0)))
final_prediction2=np.mean(df_predictions,axis=0).tolist()


In [9]:
combine_result=np.mean([final_prediction1,final_prediction2],axis=0)

In [10]:
sample_sub['click_rate']=combine_result.tolist()

In [11]:
sample_sub

In [12]:
sample_sub.to_csv('Jobathon_aug_2022_combine_ordinal_enc_and_One_hot_enc.csv',index=False)